# Project Outline

1.List topics from topic page of github 
2.List top 30 repositories for each topic with reponame, authorname, repository url, page of author as userurl, stars 
3.To save .csv file of topics and repositories info

In [12]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
url='https://github.com/topics'
base_url='https://github.com'


In [13]:
#function to get document from which we will extract all info from page

def get_document(url):
  response=requests.get(url,'html.parser')
  if(response.status_code!=200):
    raise Exception('Failed to load the page {}'.format(url))
  doc=BeautifulSoup(response.text,'html.parser') #object of BeautifulSoup //:) soup of BS4
  return doc

In [56]:
#this function will return a dataframe which will show topic title, description and url


def get_topics(url):
  doc=get_document(url)
  topic_title_tags=doc.find_all('p',{'class':'f3 lh-condensed mb-0 mt-1 Link--primary'})
  topic_desc_tags=doc.find_all('p',class_='f5 color-fg-muted mb-0 mt-1')
  topic_url_tags=doc.find_all('a',class_='d-flex no-underline')
  topic_dict={
      'title':[],
      'desc':[],
      'url':[]
  }
  
  for i in range(len(topic_title_tags)):
    topic_dict['title'].append(topic_title_tags[i].text)
    topic_dict['desc'].append(topic_desc_tags[i].text.strip())
    topic_dict['url'].append(base_url+topic_url_tags[i]['href'])
  topic_df=pd.DataFrame(topic_dict)
  return topic_df

In [15]:
# star count is in kilo format so changing it to numbers

def parse_starcount(stars):
  starcount=[]
  for i in stars:
    if(i[-1]=='k'):
      starcount.append((int(float(i[:-1])*1000)))
    else:
      starcount.append(int(i))
  return starcount


In [55]:
#function to get topics' repositories' infor like repo name, username, url of user, url of repository, stars to repo
def get_topic_repos(url):
  doc=get_document(url)
  repo_title_tags=doc.find_all('h3',class_='f3 color-fg-muted text-normal lh-condensed')
  repo_user_tags=doc.find_all('h3',class_='f3 color-fg-muted text-normal lh-condensed')
  repo_star_tags=doc.find_all('a',class_="social-count js-social-count")
  repo_url_tags=doc.find_all()
  repo_dict={
      'username':[],
      'reponame':[],
      'userurl':[],
      'repourl':[],
      'repostars':[]
  }
  for i in range(len(repo_title_tags)):
    repo_dict['username'].append(repo_title_tags[i].find_all('a')[0].text.strip())
    repo_dict['reponame'].append(repo_title_tags[i].find_all('a')[1].text.strip())
    repo_dict['userurl'].append(base_url+repo_title_tags[i].find_all('a')[0]['href'])
    repo_dict['repourl'].append(base_url+repo_title_tags[i].find_all('a')[1]['href'])
    repo_dict['repostars'].append(repo_star_tags[i].text.strip())
  repo_dict['repostars']=parse_starcount(repo_dict['repostars'])
  repo_data=pd.DataFrame(repo_dict)
  return repo_data

In [70]:
topics=get_topics(url)
topics
get_topic_repos(topics['url'][0])
get_topic_repos(topics['url'][1])
repo=get_topic_repos(get_topics(url)['url'][0])
repo.to_csv('edrepos.csv',index=None)
topics.to_csv('topics.csv',index=None)
